In [ ]:
from haskellian import either as E
import ocr_dataset as ods
import scoresheet_models as sm
from moveread.core import Core, Game, Image, glob

models = sm.ModelsCache()

In [ ]:
core = Core.read('cores-new/esna23/esna23-open-1/')
game = (await core.games.read('esna23/open/1/1')).unsafe()

In [ ]:
game.players[0].sheets

In [ ]:
await game.ocr_samples(core.blobs, models)

In [ ]:
async def remeta(core: Core):
  modified = 0
  async for i, (id, game) in core.games.items().map(E.unsafe).enumerate():
    print(f'\r{i} - modified {modified}', end='', flush=True)
    for _, sheet in game.sheets:
      model = (await models.fetch(sheet.meta.model)).unsafe()
      for img in sheet.images:
        if isinstance(img.meta, Image.OldMeta):
          img.meta = Image.Meta.from_old(img.meta, model)
          modified += 1

    (await core.games.insert(id, game)).unsafe()

In [ ]:
for core in glob('cores-new/**/*'):
  print(f'Adapting {core.base_path}...')
  await remeta(core)
  print()

In [6]:
old = ods.glob('ocr/old/**')
new = ods.glob('ocr/new/**')
ods.len(old, 'boxes'), ods.len(new, 'boxes')

(138123, 138123)

In [7]:
for i, (a, b) in enumerate(zip(ods.chain(old, 'boxes'), ods.chain(new, 'boxes'))):
  print(f'\r{i}', end='', flush=True)
  assert a == b

87796

KeyboardInterrupt: 